In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [2]:
satelite_df=pd.read_csv('satelite_df.csv')

In [3]:
satelite_test_df=pd.read_csv('satelite_test_df.csv')

In [4]:
satelite_df['date'] = pd.to_datetime(satelite_df['date'])
satelite_test_df['date'] = pd.to_datetime(satelite_test_df['date'])

In [5]:
satelite_df.drop(columns=['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',
       'B8A'],inplace=True)

In [6]:
satelite_test_df.drop(columns=['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',
       'B8A'],inplace=True)

In [7]:
satelite_df=satelite_df[['PID', 'date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07','ST_B10','LST_Day_1km', 'LST_Night_1km','ET','PET', 'EVI','NDVI','VH',
                         'VV','relativeOrbitNumber_start']]

In [8]:
satelite_test_df=satelite_test_df[['PID', 'date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07','ST_B10','LST_Day_1km', 'LST_Night_1km','ET','PET', 'EVI','NDVI','VH',
                         'VV','relativeOrbitNumber_start']]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


features = [
    'b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07',
    'ST_B10', 'LST_Day_1km', 'LST_Night_1km',
    'ET', 'PET', 'EVI', 'NDVI', 'VH', 'VV',
    'relativeOrbitNumber_start'
]


satelite_df_subset = satelite_df[features].dropna(how='all')


corr = satelite_df_subset.corr()


plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Heatmap of Selected Features")
plt.show()


# 1

In [11]:
satelite_bands=pd.DataFrame()

In [36]:
satelite_test_bands=pd.DataFrame()

In [13]:
satelite_bands[['PID','date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07']]=satelite_df[['PID','date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07']]

In [37]:
satelite_test_bands[['PID','date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07']]=satelite_test_df[['PID','date','b01', 'b02', 'b03', 'b04', 'b05', 'b06',
       'b07']]

In [18]:
satelite_bands.isnull().sum()

PID           0
date          0
b01     1804093
b02     1804093
b03     1804093
b04     1817448
b05     2395625
b06     2395625
b07     2307544
dtype: int64

In [19]:
len(satelite_bands)

8485722

In [30]:

satelite_bands['date'] = pd.to_datetime(satelite_bands['date'])

satelite_bands['2month_bin'] = satelite_bands['date'].dt.to_period('M').apply(
    lambda p: pd.Period(f"{p.year}-{(p.month - 1)//2*2 + 1:02d}", freq='M')
)

satelite_bands_agg = satelite_bands.groupby(['PID', '2month_bin']).mean(numeric_only=True).reset_index()


wide = satelite_bands_agg.pivot(index='PID', columns='2month_bin')


wide.columns = [f"{band}_{period}" for band, period in wide.columns]
wide = wide.reset_index()


for col in wide.columns:
    if col == 'PID':
        continue
    wide[col] = wide[col].fillna(wide[col].mean())


In [38]:

satelite_test_bands['date'] = pd.to_datetime(satelite_test_bands['date'])

satelite_test_bands['2month_bin'] = satelite_test_bands['date'].dt.to_period('M').apply(
    lambda p: pd.Period(f"{p.year}-{(p.month - 1)//2*2 + 1:02d}", freq='M')
)


satelite_test_bands_agg = satelite_test_bands.groupby(['PID', '2month_bin']).mean(numeric_only=True).reset_index()


wide_test = satelite_test_bands_agg.pivot(index='PID', columns='2month_bin')

wide_test.columns = [f"{band}_{period}" for band, period in wide_test.columns]
wide_test = wide_test.reset_index()


for col in wide_test.columns:
    if col == 'PID':
        continue
    wide_test[col] = wide_test[col].fillna(wide_test[col].mean())


In [39]:
wide_test['PID'].nunique()

2418

In [40]:
wide.to_csv('bands.csv', index=False)

In [41]:
wide_test.to_csv('bands_test.csv', index=False)